In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
from data_setup import ZRI_format

In [3]:
#Load Data
ZRI_MF = pd.read_pickle('./pickles/ZRI_filtered.p')
ACS = pd.read_pickle('./acs_data/ACS.p')
crime = pd.read_pickle('./pickles/crime.p')
dominant_county = pd.read_pickle('./pickles/dominant_county_zip.p')
weather = pd.read_pickle('./pickles/weather.p')
cbp = pd.read_pickle('./pickles/cbp_zip.p')

In [6]:
%%time
ZRI_new = ZRI_format(ZRI_MF, time_unit = 'Month', window_size = 12, future_time = 36)

Wall time: 10.5 s


In [16]:
acs_lag = 2
ACS = ACS.assign(year_avail = (ACS.year.astype(int) + acs_lag).astype(str))

In [19]:
cbp_lag = 2
cbp = cbp.assign(year_avail = (cbp.Year.astype(int) + cbp_lag).astype(str))

In [22]:
#ACS
ZRI = ZRI_new.merge(ACS,how = 'left',left_on = ['ZipCode','Predict_Year'], 
                                              right_on = ['ZipCode','year_avail'])

In [23]:
#Crime
ZRI = ZRI.assign(dominant_county = ZRI.ZipCode.apply(lambda x: dominant_county[x]))
ZRI = ZRI.merge(crime[['crime_rate_per_100000','county_fips_code']],how = 'left',
          left_on = 'dominant_county',right_on = 'county_fips_code').drop('county_fips_code',axis = 1)

In [28]:
cbp

,ZIP,Year,num_businesses,num_employees,total_payroll,year_avail
11996,99501,2011,1611.094373,27553.001726,1.587804e+06,2013
11997,99501,2012,1338.077782,22806.900145,1.354551e+06,2014
11998,99501,2013,1346.232382,23827.953500,1.411982e+06,2015
11999,99501,2014,1374.291342,23288.321370,1.439466e+06,2016
12000,99501,2015,1384.938720,23541.896605,1.469234e+06,2017
...,...,...,...,...,...,...
562498,54901,2014,657.081589,15421.271869,7.318644e+05,2016
562499,54901,2015,644.583496,15286.337208,7.662851e+05,2017
562500,54901,2016,662.749501,15705.154845,7.941442e+05,2018
562501,54901,2017,669.231182,15817.621251,8.052528e+05,2019


In [27]:
#CBP
ZRI = ZRI.merge(cbp[['ZIP','num_businesses','num_employees','total_payroll','year_avail']],
                    how = 'left',
                left_on = ['ZipCode','Predict_Year'],
               right_on = ['ZIP',''])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

In [ ]:
#Columns to use in the final analysis
zip_columns = ['geo_id','unemployed_pop','white_pop','vacant_housing_units','total_pop','worked_at_home',
               'poverty','percent_income_spent_on_rent','occupied_housing_units',
               'median_year_structure_built','median_age','married_households','masters_degree',
               'male_pop','female_pop','income_per_capita','housing_units','employed_pop','black_pop',
               'asian_pop','amerindian_pop','graduate_professional_degree']

In [ ]:
#Convert columns to percentage
#Columns to divide by total population
pop_columns = ['unemployed_pop','white_pop','masters_degree',
               'graduate_professional_degree','employed_pop','black_pop',
               'asian_pop','amerindian_pop','poverty','worked_at_home']

#Columns to divide by total housing units
house_columns = ['vacant_housing_units','occupied_housing_units']

#Division
ZRI.loc[:,pop_columns] = ZRI[pop_columns].div(ZRI['total_pop'], axis = 0)
ZRI.loc[:,house_columns] = ZRI[house_columns].div(ZRI['housing_units'], axis = 0)

In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from collections import defaultdict

In [ ]:
#Find feature columns
full_feature_columns = [x for x in ZRI.columns if 'minus' in x] +\
                                                     zip_columns +\
                                                   ['crime_rate_per_100000']

In [ ]:
ZRI_feature_columns = [x for x in ZRI.columns if 'minus' in x]

In [ ]:
#Train test split, test data is above a given year
test_year = 2019
data_4_model = ZRI[full_feature_columns + ['Target_ZRI','Year']].dropna()
training_data = data_4_model[data_4_model.Year < test_year]
test_data = data_4_model[data_4_model.Year >= test_year]

In [ ]:
data_4_model[full_feature_columns]

In [ ]:
X_train_full, y_train_full = training_data[full_feature_columns], training_data['Target_ZRI']

In [ ]:
X_test_full, y_test_full = test_data[full_feature_columns], test_data['Target_ZRI']

In [ ]:
lasso_params = {'alpha': [0.1,1,2,3,4,5]}
lasso_grid = GridSearchCV(Lasso(), param_grid=lasso_params)
lasso_model = make_pipeline(StandardScaler(),lasso_grid)
rf_params = {'max_depth' : [None,10]}
rf_model = GridSearchCV(RandomForestRegressor(n_jobs = -1),param_grid= rf_params)

In [ ]:
rf_model.fit(X_train_full,y_train_full)

In [ ]:
rf_coef_importance = pd.Series(dict(zip(X_train_full.columns, rf_model.best_estimator_.feature_importances_))).sort_values()

In [ ]:
rf_model.score(X_test_full,y_test_full), rf_model.score(X_train_full,y_train_full)

In [ ]:
rf_coef_importance.loc[[x for x in X_train_full.columns if ('minus' not in x) and (x != 'geo_id')]].sort_values().plot(kind = 'bar')

In [ ]:
lasso_model.fit(X_train_full,y_train_full)

In [ ]:
coefficients = pd.Series(dict(zip(X_train_full.columns, lasso_model.named_steps.gridsearchcv.best_estimator_.coef_))).sort_values()

In [ ]:
coefficients.loc[[x for x in X_train_full.columns if ('minus' not in x) and (x != 'geo_id')]].sort_values().plot(kind='bar')

In [ ]:
lasso_model.score(X_test_full,y_test_full), lasso_model.score(X_train_full, y_train_full)

In [ ]:
prediction_error_full = final_test_data['Target_ZRI'] - lr_full.predict(final_test_data[full_feature_columns])

In [ ]:
prediction_error_ZRI = final_test_data['Target_ZRI'] - lr_zri.predict(final_test_data[ZRI_feature_columns])

In [ ]:
prediction_error_full.describe(), prediction_error_ZRI.describe()

In [ ]:
prediction_error = final_test_data['Target_ZRI'] - lr.predict(final_test_data[feature_columns])

In [ ]:
prediction_error.describe()

In [ ]:
lr.coef_, lr.alpha_

In [ ]:
lr.coef_

In [ ]:
plt.boxplot(errors.values())

In [ ]:
pd.DataFrame(errors).describe()